In [36]:

import pandas as pd
import numpy as np
import lightgbm as lgb
import sys
import math
import gc
import sklearn.metrics as skl_metrics
from sklearn.metrics import mean_squared_error

# import math
# import sklearn.metrics as skl_metrics
# from sklearn.metrics import mean_squared_error

from logging import StreamHandler, DEBUG, Formatter, FileHandler, getLogger

logger = getLogger(__name__)

pd.options.mode.chained_assignment = None  # default='warn'

DIR = '../logs/'

log_fmt = Formatter('%(asctime)s %(name)s %(lineno)d [%(levelname)s]\
    [%(funcName)s] %(message)s ')
handler = StreamHandler()
handler.setLevel('INFO')
handler.setFormatter(log_fmt)
logger.addHandler(handler)

handler = FileHandler(DIR + 'train.py.log', 'a')
handler.setLevel(DEBUG)
handler.setFormatter(log_fmt)
logger.setLevel(DEBUG)
logger.addHandler(handler)

logger.info('start')

2018-01-04 14:08:18,155 __main__ 35 [INFO]    [<module>] start 
2018-01-04 14:08:18,155 __main__ 35 [INFO]    [<module>] start 
2018-01-04 14:08:18,155 __main__ 35 [INFO]    [<module>] start 
2018-01-04 14:08:18,155 __main__ 35 [INFO]    [<module>] start 
2018-01-04 14:08:18,155 __main__ 35 [INFO]    [<module>] start 
2018-01-04 14:08:18,155 __main__ 35 [INFO]    [<module>] start 
2018-01-04 14:08:18,155 __main__ 35 [INFO]    [<module>] start 


In [37]:
train_out = pd.read_pickle('../data/storeitem_train_1s.p')
val_out = pd.read_pickle('../data/storeitem_val_1s.p')
X_test_out = pd.read_pickle('../data/storeitem_test_1s.p')

item_train_out = pd.read_pickle('../data/item_train_1s.p')
item_val_out = pd.read_pickle('../data/item_val_1s.p')
item_X_test_out = pd.read_pickle('../data/item_test_1s.p')

df_test = pd.read_csv(
    "../input/test_1s.csv", usecols=[0, 1, 2, 3, 4],
    dtype={'onpromotion': bool},
    parse_dates=["date"]  # , date_parser=parser
).set_index(
    ['store_nbr', 'item_nbr', 'date']
)


items = pd.read_csv(
    "../input/items.csv",
).set_index("item_nbr")
items = items.reindex(train_out.item_nbr)

items_val = pd.read_csv(
    "../input/items.csv",
).set_index("item_nbr")
items_val = items_val.reindex(val_out['item_nbr'])


In [38]:
param_1 = '1s'

param_1 = '1s'

In [39]:


items = pd.read_csv(
    "../input/items.csv",
).set_index("item_nbr")
items = items.reindex(train_out.item_nbr)

items_val = pd.read_csv(
    "../input/items.csv",
).set_index("item_nbr")
items_val = items_val.reindex(val_out['item_nbr'])

logger.info('Load data successful')

###############################################################################
# Merge item features with item_store features

del train_out["index"]
del item_train_out["index"]

train_out = pd.merge(train_out, item_train_out, how='inner', on=['item_nbr','date'])
val_out = pd.merge(val_out, item_val_out, how='inner', on=['item_nbr','date'])
X_test_out = pd.merge(X_test_out, item_X_test_out, how='inner', on=['item_nbr','date'])

###############################################################################
logger.info('Preparing traing dataset...')

all_columns = train_out.columns.tolist()

y_columns = ['day'+str(i) for i in range(1, 17)]
x_columns = [item for item in all_columns if item not in y_columns]

features_all = x_columns
features_all.remove("date") 
features_all.remove("item_nbr") 
features_all.remove("store_nbr") 

X_train_out = train_out[x_columns]
X_val_out = val_out[x_columns]

y_train = train_out[y_columns].values
y_val = val_out[y_columns].values

X_train_allF = X_train_out[features_all]
X_val_allF = X_val_out[features_all]
X_test_allF = X_test_out[features_all]

#del train_out, val_out
#del X_train_out, X_val_out, X_test_out
gc.collect()

##########################################################################
logger.info('Training and predicting models...')

params = {
    'num_leaves': 31,
    'objective': 'regression',
    'min_data_in_leaf': 300,
    'learning_rate': 0.1,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 2,
    'metric': 'l2',
    'num_threads': 4
}

MAX_ROUNDS = 500
val_pred = []
test_pred = []
cate_vars = []


train_week_2017 = 7
if param_1 != "val":
    train_week_2017 = 9
    
features_all = X_train_allF.columns.tolist()


2018-01-04 14:08:18,421 __main__ 13 [INFO]    [<module>] Load data successful 
2018-01-04 14:08:18,421 __main__ 13 [INFO]    [<module>] Load data successful 
2018-01-04 14:08:18,421 __main__ 13 [INFO]    [<module>] Load data successful 
2018-01-04 14:08:18,421 __main__ 13 [INFO]    [<module>] Load data successful 
2018-01-04 14:08:18,421 __main__ 13 [INFO]    [<module>] Load data successful 
2018-01-04 14:08:18,421 __main__ 13 [INFO]    [<module>] Load data successful 
2018-01-04 14:08:18,421 __main__ 13 [INFO]    [<module>] Load data successful 
2018-01-04 14:08:18,496 __main__ 26 [INFO]    [<module>] Preparing traing dataset... 
2018-01-04 14:08:18,496 __main__ 26 [INFO]    [<module>] Preparing traing dataset... 
2018-01-04 14:08:18,496 __main__ 26 [INFO]    [<module>] Preparing traing dataset... 
2018-01-04 14:08:18,496 __main__ 26 [INFO]    [<module>] Preparing traing dataset... 
2018-01-04 14:08:18,496 __main__ 26 [INFO]    [<module>] Preparing traing dataset... 
2018-01-04 14:08:

In [40]:

train_out["date"] = pd.to_datetime(train_out["date"])
if ((param_1 == "val") or (param_1 == "1s")):
    train_out = train_out.loc[train_out["date"] < '2017-07-19', ]


In [41]:
i = 0 

print("=" * 70)
logger.info("Step %d" % (i+1))
print("=" * 70)
features_t = features_all.copy()

for j in range(16):
    if j != i:
        features_t.remove('ly_1d_d{}'.format(j))
        features_t.remove('l2y_1d_d{}'.format(j))

for j in range(7):
    if j != i%7:
        features_t.remove('dow_1_{}_mean'.format(j))
        features_t.remove('dow_4_{}_mean'.format(j))
        features_t.remove('dow_8_{}_mean'.format(j))
        features_t.remove('dow_13_{}_mean'.format(j))
        features_t.remove('dow_26_{}_mean'.format(j))
        features_t.remove('dow_52_{}_mean'.format(j))
        features_t.remove('dow_ly3w_{}_mean'.format(j))
        features_t.remove('dow_ly8w_{}_mean'.format(j))

        features_t.remove('item_dow_4_{}_mean'.format(j))
        features_t.remove('item_dow_13_{}_mean'.format(j))
        features_t.remove('item_dow_26_{}_mean'.format(j))
        features_t.remove('item_dow_52_{}_mean'.format(j))


X_train = X_train_allF[features_t]
X_val = X_val_allF[features_t]
X_test = X_test_allF[features_t]

dtrain = lgb.Dataset(
    X_train, label=y_train[:, i],
    categorical_feature=cate_vars,
    weight=pd.concat([items["perishable"]]) * 0.25 + 1
)

if ((param_1 == "val") or (param_1 == "1s")):
    dval = lgb.Dataset(
         X_val, label=y_val[:, i], reference=dtrain,
         weight=items_val["perishable"] * 0.25 + 1,
         categorical_feature=cate_vars)

    bst = lgb.train(
        params, dtrain, num_boost_round=MAX_ROUNDS,
        valid_sets=[dtrain, dval], early_stopping_rounds=50,
        verbose_eval=100
    )

else:

    bst = lgb.train(
        params, dtrain, num_boost_round=MAX_ROUNDS,
        valid_sets=[dtrain], early_stopping_rounds=50, verbose_eval=100
    )

logger.info("\n".join(("%s: %.2f" % x) for x in sorted(
    zip(X_train.columns, bst.feature_importance("gain")),
    key=lambda x: x[1], reverse=True
)))

test_pred.append(bst.predict(
    X_test, num_iteration=bst.best_iteration or MAX_ROUNDS))

if ((param_1 == "val") or (param_1 == "1s")):
    val_pred.append(bst.predict(
        X_val, num_iteration=bst.best_iteration or MAX_ROUNDS))

del X_train, y_train
del dtrain
gc.collect()


2018-01-04 14:08:18,808 __main__ 4 [INFO]    [<module>] Step 1 
2018-01-04 14:08:18,808 __main__ 4 [INFO]    [<module>] Step 1 
2018-01-04 14:08:18,808 __main__ 4 [INFO]    [<module>] Step 1 
2018-01-04 14:08:18,808 __main__ 4 [INFO]    [<module>] Step 1 
2018-01-04 14:08:18,808 __main__ 4 [INFO]    [<module>] Step 1 
2018-01-04 14:08:18,808 __main__ 4 [INFO]    [<module>] Step 1 
2018-01-04 14:08:18,808 __main__ 4 [INFO]    [<module>] Step 1 
/home/zyp/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1030: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 50 rounds.
[100]	training's l2: 0.249349	valid_1's l2: 0.239901
[200]	training's l2: 0.230128	valid_1's l2: 0.222061
[300]	training's l2: 0.214812	valid_1's l2: 0.20676
[400]	training's l2: 0.201696	valid_1's l2: 0.193711


2018-01-04 14:08:23,011 __main__ 61 [INFO]    [<module>] mean_7_2017: 54966.97
mean_14_2017: 53449.86
mean_42_2017: 17466.03
mean_21_2017: 6195.68
promo_0: 4786.38
dow_26_0_mean: 4695.80
mean_3_2017: 3515.54
dow_8_0_mean: 3297.29
dow_13_0_mean: 2820.91
dow_52_0_mean: 2275.05
day_1_2017: 1646.07
mean_91_2017: 1390.68
dow_4_0_mean: 1258.72
mean_30_2017: 1100.88
mean_182_2017: 1073.54
item_mean_7_2017: 968.49
promo_14_2017: 877.60
mean_60_2017: 823.54
dow_1_0_mean: 740.27
item_dow_4_0_mean: 723.10
mean_ly_n16d_2017: 723.00
item_mean_21_2017: 662.46
dow_ly3w_0_mean: 651.98
item_day_1_2017: 651.14
promo_140_2017: 649.65
mean_ly_7_2017: 649.41
dow_ly8w_0_mean: 620.89
item_dow_52_0_mean: 604.68
item_dow_13_0_mean: 594.33
mean_140_2017: 587.80
item_mean_91_2017: 587.37
item_mean_42_2017: 580.70
item_mean_364_2017: 569.88
mean_ly_30_2017: 563.48
item_dow_26_0_mean: 556.83
mean_ly_14_2017: 551.38
mean_364_2017: 496.60
item_mean_182_2017: 482.07
l2y_1d_d0: 457.39
mean_ly_21_2017: 449.35
promo_60_

[500]	training's l2: 0.189965	valid_1's l2: 0.181778


139